In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-05-08


Found rundate 2024-05-07 10:00:03 < 2024-05-08 at commit f90047f0; returning commit 0c28e96


'2024-05-08 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-05-08 10:00:02, NJSP was reporting 214 YTD deaths
Current YTD Deaths (2025-05-08 10:00:05-04:00): 175
Previous year YTD Deaths (adjusted; 2024-05-08 10:00:02): 212.9148013755924
Projected 2025 total: 617.7482564046447
-17.8% change


7298ecbabc822cda4fb9b11f283fabbbb93cab71: FAUQStats cache miss: 2024, Wed May 08 10:00:02 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13156,06,Cumberland,0605,Fairfield Twsp,NaN,State Highway 49 MP 27.5,1.0,1.0,0.0,0.0,0.0,49,NaN,2024-05-01 09:48:00-04:00
13157,15,Ocean,1507,Toms River,NaN,State Highway 166,1.0,0.0,0.0,1.0,0.0,166,NaN,2024-05-01 20:25:00-04:00
13158,06,Cumberland,0613,Upper Deerfield Tws,NaN,State Highway 77 MP 2.5,1.0,1.0,0.0,0.0,0.0,77,NaN,2024-05-01 20:34:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,8,0,3,0,10
Bergen,5,1,2,0,8
Burlington,8,3,3,0,13
Camden,10,2,5,0,16
Cape May,2,0,0,0,2
Cumberland,3,2,2,0,6
Essex,4,1,7,1,13
Gloucester,8,2,1,0,9
Hudson,0,0,2,0,2


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

34.9% through the year, 65.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,8,31,8
Bergen,driver,7,14,5
Burlington,driver,12,32,8
Camden,driver,7,19,10
Cape May,driver,1,8,2
...,...,...,...,...
Salem,crashes,4,14,6
Somerset,crashes,3,18,6
Sussex,crashes,2,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,44,1,31,1,15
Bergen,26,1,11,3,11
Burlington,40,1,26,4,11
Camden,34,0,24,4,9
Cape May,13,0,11,0,1
Cumberland,23,0,14,9,9
Essex,51,2,17,6,33
Gloucester,33,0,24,8,6
Hudson,14,0,5,0,10


In [9]:
projected.to_csv(PROJECTED_CSV)